# CSE547 - Colab 4
## Collaborative Filtering

Adapted from Stanford's CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=93c5384bf04d4f854e167c9302df1f14b13c356a0c1673ed26ca8c11841dcfc9
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-28 01:35:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.233.35.85, 34.192.215.160, 54.174.137.247, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.233.35.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  39.1MB/s    in 0.3s    

2020-04-28 01:35:18 (39.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://616c5a2b.ngrok.io


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [0]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [9]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [42]:
items.printSchema()
items.take(1)

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



[Row(item_id=1, movie='Toy Story (1995)')]

### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [13]:
print(items.count())
print(training.count())

1682
80000


Using the training set, train models with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

Model 1: 

*   maximal iteration = 10
*   **rank = 10**

*   regularization = 0.1
*   Drop rows with a NaN value in rating



In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [0]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
  maxIter=10, 
  regParam=0.1, 
  rank=10,
  userCol="user_id", 
  itemCol="item_id", 
  ratingCol="rating",
  coldStartStrategy="drop"
  )
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(np.round(rmse,3)))





To experiment with different values of rank, repeat the process of computing RMSE for the following models (Feel free to just edit the code you wrote above):

Model 2: 

*   Model 1 with **rank = 5**


In [31]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als2 = ALS(
  maxIter=10, 
  regParam=0.1, 
  rank=10,
  userCol="user_id", 
  itemCol="item_id", 
  ratingCol="rating",
  coldStartStrategy="drop"
  )
model2 = als2.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions2 = model2.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse2 = evaluator.evaluate(predictions2)
print("Root-mean-square error = " + str(np.round(rmse2,3)))





Root-mean-square error = 0.939


Model 3: 

*   Model 1 with **rank = 100**

In [63]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als3 = ALS(
  maxIter=10, 
  regParam=0.1, 
  rank=100,
  userCol="user_id", 
  itemCol="item_id", 
  ratingCol="rating",
  coldStartStrategy="drop"
  )
model3 = als3.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions3 = model3.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

rmse3 = evaluator.evaluate(predictions3)
print("Root-mean-square error = " + str(np.round(rmse3,3)))





Root-mean-square error = 0.93


Now experiment with different regularization values using Model 3 (**rank = 100**). Which regularization gives the lowest RMSE?  (Feel free to just edit the code you wrote above)

*   1
* 0.3
* 0.1
* 0.03
* 0.01



In [36]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
reg = [1,0.3,0.1,0.03,0.01]
rmse4 = []

for r in reg:
  als4 = ALS(
    maxIter=10, 
    regParam=r, 
    rank=100,
    userCol="user_id", 
    itemCol="item_id", 
    ratingCol="rating",
    coldStartStrategy="drop"
    )
  model4 = als4.fit(training)

  # Evaluate the model by computing the RMSE on the test data
  predictions4 = model4.transform(test)
  evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

  rmse4.append(evaluator.evaluate(predictions4))

print("Root-mean-square error = ")
for i in range(len(reg)):
  print(reg[i],rmse4[i])







Root-mean-square error = 
1 1.3793451688357117
0.3 0.9906362000782779
0.1 0.9300177502954377
0.03 0.9731068978878368
0.01 1.1324223700653058


At this point, you can use the trained model to produce the top-K recommendations for each user. 

Using model 3 (**rank 100**, **regularization 0.1**) find the most frequent #1 movie recommendation across all users. You may find it easier to convert the recommendation output to a Pandas Dataframe. Which movie is it?

Hint: One approach is described on GradeScope.

In [0]:
# Generate top 10 movie recommendations for each user
userRecs = model3.recommendForAllUsers(1)

In [67]:
df = userRecs.toPandas()
df['item_id'] = [np.int64(r[0][0]) for r in df['recommendations']]
moviesPD = items.toPandas()
moviesPD['item_id'] = np.int64(moviesPD['item_id'])

df=df.merge(moviesPD, on='item_id')


print(df['movie'].mode())

0    Pather Panchali (1955)
dtype: object


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!